# Assignment4 Week4

# Employee attrition


In [34]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import math

from sklearn.metrics import r2_score
import sklearn as sk
from sklearn.model_selection import train_test_split 

from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz


In [35]:
df = pd.read_csv('Employee.csv')
df.head(50)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
5,32,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,8,...,3,80,0,8,2,2,7,7,3,6
6,59,No,Travel_Rarely,1324,Research & Development,3,3,Medical,1,10,...,1,80,3,12,3,2,1,0,0,0
7,30,No,Travel_Rarely,1358,Research & Development,24,1,Life Sciences,1,11,...,2,80,1,1,2,3,1,0,0,0
8,38,No,Travel_Frequently,216,Research & Development,23,3,Life Sciences,1,12,...,2,80,0,10,2,3,9,7,1,8
9,36,No,Travel_Rarely,1299,Research & Development,27,3,Medical,1,13,...,2,80,2,17,3,2,7,7,7,7


In [36]:
dummies = pd.get_dummies(df['Gender'])
dummies.head()

,Female,Male
0,1,0
1,0,1
2,0,1
3,1,0
4,0,1


In [37]:
df = pd.concat([df, dummies], axis=1) 
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Female,Male
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,0,8,0,1,6,4,0,5,1,0
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,1,10,3,3,10,7,1,7,0,1
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,0,7,3,3,0,0,0,0,0,1
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,0,8,3,3,8,7,3,0,1,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,1,6,3,3,2,2,2,2,0,1


In [38]:
df_subset = df[['Age', 'DailyRate', 'Female', 'Attrition', 'Education', 'DistanceFromHome', 'TotalWorkingYears', 'YearsAtCompany']]
df_subset.head(10)

,Age,DailyRate,Female,Attrition,Education,DistanceFromHome,TotalWorkingYears,YearsAtCompany
0,41,1102,1,Yes,2,1,8,6
1,49,279,0,No,1,8,10,10
2,37,1373,0,Yes,2,2,7,0
3,33,1392,1,No,4,3,8,8
4,27,591,0,No,1,2,6,2
5,32,1005,0,No,2,2,8,7
6,59,1324,1,No,3,3,12,1
7,30,1358,0,No,1,24,1,1
8,38,216,0,No,3,23,10,9
9,36,1299,0,No,3,27,17,7


In [39]:
y = df_subset['Attrition'] #the indepent variable 
X = df_subset[['Age', 'DailyRate', 'Female', 'Education', 'DistanceFromHome', 'TotalWorkingYears', 'YearsAtCompany']] #the depentent variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) #30- test/70- training relation  

In [40]:
model = RandomForestClassifier(max_depth=5, random_state=1)
model = model.fit(X, y)
y_test_pred = model.predict(X_test) #the predicted values
cm = confusion_matrix(y_test, y_test_pred) #creates a "confusion matrix" on the test set
cm

/Users/quintennaayen/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([[363,   1],
       [ 66,  11]])

In [41]:
conf_matrix = pd.DataFrame(cm, index=['NoAttrition_real', 'YesAttrition_real'], columns = ['NoAttrition_predict', 'YesAttrition_predict']) 
conf_matrix

,NoAttrition_predict,YesAttrition_predict
NoAttrition_real,363,1
YesAttrition_real,66,11


The employer's confusion matrix dataset results in the following results: 
- 363 times it was well predicted that an employer had dropped out. only once was this prediction incorrect. 
- 66 times it was incorrectly predicted No attrition in stead of Yes Attrition.






In [42]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

          No       0.85      1.00      0.92       364
         Yes       0.92      0.14      0.25        77

    accuracy                           0.85       441
   macro avg       0.88      0.57      0.58       441
weighted avg       0.86      0.85      0.80       441



**Accuracy**

Accuracy = Correct/Total * 100 
374/441 * 100 = 84,8 %

**Precision** 

Precision talks about how precise/accurate your model is out of those predicted Attrition, how many of them are actual well predicted.

**Recall**

So Recall actually calculates how many of the Actual Yes our model capture through labeling it as Yes. Applying the same understanding, we know that Recall shall be the model metric we use to select our best model when there is a high cost associated with False No.

No Attrition:
precision = .85
recall = 1.00

Yes Attrition:
precision = 0.92
recall = 0.14